In [2]:
import re
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import wordnet as wn
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from IPython.display import clear_output

In [24]:
# ambil stopwords
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
stopword = factory.create_stop_word_remover()

# stemmer
factory2 = StemmerFactory()
stemmer = factory2.create_stemmer()

def preprocessing(semua_kalimat):
    print(semua_kalimat)
    panjang = len(semua_kalimat)
    i=1
    kalimat = []
    for sentence in semua_kalimat:
        # 4
        sentence = stemmer.stem(sentence)
        # 5
        sentence = stopword.remove(sentence)
        # 6 & 7
#         tokens = re.sub(r"[^a-z0-9]+", " ", sentence.lower()).split()
        kalimat.append(sentence)
        clear_output(wait=True)
        print(sentence)
        print((i/panjang)*100,"%")
        i+=1
    return kalimat

In [110]:
print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [210]:
''' ABAIKAN, ini cuma buat mindah data '''
def pecahkalimat(sourcefile,destfile):
#     open(destfile, 'w').close()
    f = open(sourcefile)
    i=1
    kalimat = []
    with open(destfile, "a") as myfile:
        for sentence in f:
            kal = sentence.split()
            if "\n" in kal:
                kal.remove("\n")
            
            kal = ' '.join(kal)
            kal = kal.strip()
            lab = kal[-1:]
            kal = re.sub(r"[^a-z0-9]+", " ", kal.lower()).strip()
            #print(kal)
            if len(kal) != 0:
                myfile.write(kal[:-2])
#                 myfile.write(kal)
                myfile.write(",")
                myfile.write(lab)
                myfile.write("\n")

pecahkalimat("bulan.txt","train_bulan.csv")

In [211]:
# input file
df = pd.read_csv("train_bulan.csv")
labels = df["label"]
# preprocess
semkal = preprocessing(df['kalimat'])

tanda tanda matahari bulan bintang bintang di bumi bangsa bangsa akan takut bingung hadap deru gelora laut
100.0 %


In [212]:
kalimat = []
i=0
with open("bulan_pre.csv", "a") as myfile:
    for sentence in semkal:
        myfile.write(sentence)
        myfile.write(",")
        myfile.write(str(labels[i]))
        myfile.write("\n")
        i+=1

In [286]:
# settings tf-idf
tfidf = TfidfVectorizer(sublinear_tf=False, min_df=7, analyzer='word', ngram_range=(1,2))

In [293]:
df = pd.read_csv("motor_pre.csv")
# tf-idf
features = tfidf.fit_transform(df["kalimat"].values.astype('U')).toarray()
features_name = tfidf.vocabulary_
labels = df["label"]
print("terdapat",len(features_name),"kolom fitur")

terdapat 432 kolom fitur


In [294]:
# print(features_name)

In [295]:
''' Training Data '''
x=[]
y=[]
for i in range(0,593):
    x.append(features[i])
    y.append(labels[i])

''' Testing Data '''
a=[]
b=[]
akal=[]
for i in range(593,701):
    a.append(features[i])
    b.append(labels[i])
    akal.append(df['kalimat'][i])
# for i in range(381,462):
#     a.append(features[i])
#     b.append(labels[i])

In [296]:
''' The important part '''
# ''' SVM classifier ''' 
# model
model_svm = LinearSVC()
# training
model_svm.fit(x, y)
# predict / testing
pred = model_svm.predict(a)

In [297]:
# ''' buat nampilin hasil doang & confussion matrix '''
x=0
satu=0
dua=0
fs=0
fd=0

for i in pred:
#     print(i, b[x])
    if i == b[x] and i == 1:
        satu+=1
    elif i == b[x] and i == 2:
        dua+=1
    elif i == 2:
        fd+=1
    else:
        fs+=1
    x+=1

print("Akurasi:",(satu+dua)/len(pred),"\n")
print(satu,fs)
print(fd,dua)

Akurasi: 0.9537037037037037 

63 3
2 40


In [222]:
# x=1
# for i in b:
#     print(x,i)
#     x+=1

In [292]:
# nampilin tabel hasil prediksi, testing dari oxford dictionary
# df = pd.read_csv("train_bulan.csv")
aa=[]
p=[]
l=[]
# i=264
for i in range(0, 87):
    aa.append(akal[i])
#     aa.append(df["kalimat"][i])
    p.append(pred[i])
    l.append(b[i])
    i+=1
    
out = pd.DataFrame(aa)
out['lab'] = l
out['pred'] = p

out.to_csv('hasil_bulan.csv')
pd.set_option('max_colwidth',1000)
pd.options.display.max_rows = 110
display(out)

,0,lab,pred
0,lama 4 bulan,1,1
1,bulan mobil jadi,1,1
2,ada di teater bulan,1,2
3,tiga hari tanggal tujuh bulan juni,1,1
4,tiket sedia satu bulan,1,2
5,musim hujan pada bulan juni juli,1,1
6,festival sekolah pada bulan oktober,1,1
7,telah milik layan sejak bulan lal,1,1
8,tidak sabar tunggu bulan purnama,2,2
9,sudah lihat 13 bulan lamanya1,1,1


In [ ]:
# nampilin tabel hasil prediksi, testing dari sentence.yourdictionary.com
i=46
z=[]
kal = pd.read_csv('train2.csv')
for row in kal["kalimat"]:
    z.append(pred[i])
    i+=1
outkal = pd.DataFrame(kal["kalimat"])
outkal['label'] = kal["label"]
outkal['pred'] = z

pd.set_option('max_colwidth',1000)
pd.options.display.max_rows = 100
display(outkal)

In [90]:
kal="aku asafdfa 2"
print(kal[:-2])

aku asafdfa
